<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 1 / NOTE 4*

---

### 실험 기록 툴(wandb 등)에 대해 알아보기


- **실험 기록을 하는 이유는 무엇일까요?**

다양한 하이퍼파라미터를 변경해가면서 장기적으로 실험을 진행하다보면 점점 결과를 관리하기가 어려워집니다.<br/>
"어떤 파라미터 조합이 제일 좋았지?", "어제 했던 결과와 차이가 있었던가?" 와 같은 의문을 품게 되죠.<br/>
비록 노트북(`.ipynb`)이 어느정도 출력물을 기록하기는 있지만 모든 실험 결과를 관리하기엔 적절하지 않습니다.

**Comet.ml, Weights and Biases(`wandb`)** 등은 이러한 문제를 해결하기 위해 등장한 실험 기록 도구입니다.<br/>
이런 실험 기록 도구는 실험 결과를 실시간으로 기록하고 **코드와 결과값을 보관**해주며,<br/>
실험 결과를 원하는 기준대로 언제든지 **시각화하여 모델의 성능을 확인**할 수 있도록 도와줍니다.<br/>
매 Epoch이 끝날 때마다 데이터가 해당 툴에 보내지기 때문에 **모델이 수렴하고 있는지도 확인**할 수 있습니다.

이번 시간에는 Weights and Biases(`wandb`)를 활용하여 실험 기록을 수행해보도록 하겠습니다.

#### Wandb 이용하기

- 설치 및 회원가입<br/>
먼저 다음 셀을 실행하기 전에 터미널에서 `wandb`에 로그인이 되있어야 합니다. 

    ```zsh
# 아래의 커맨드를 실행합니다
wandb.login
```
구체적인 방법은 Weights and Biases의 [QuickStart](https://docs.wandb.com/quickstart)를 참고해주시면 되겠습니다.

In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
# group, project 변수를 설정합니다. 반복되는 이름이 많기 때문에 변수로 설정하여 사용하면 편리합니다.
wandb_project = "review"
wandb_group = ""

In [ ]:
# !git clone http://github.com/wandb/tutorial

In [ ]:
# !cd tutorial; pip install --upgrade -r requirements.txt;

In [ ]:
!wandb login 6a1f7dd199ef2c241cc2dafb7ae52925d6de7385

In [ ]:
import numpy
import pandas as pd
#from tensorflow import keras
#from tensorflow.python import keras
#from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV

In [ ]:
# !python -c "import keras; print(keras.__version__)"

In [ ]:
wandb.init(project=wandb_project)  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.
#wandb.init(project=wandb_project, entity=wand_group) 

# 데이터 및 하이퍼파라미터 설정 
X =  x_train
y =  y_train

inputs = X.shape[1]
wandb.config.epochs = 50
wandb.config.batch_size = 10

# 모델을 구축합니다
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# 모델을 컴파일 합니다
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# 모델을 학습합니다
model.fit(X, y, 
          validation_split=0.3, 
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
         )

In [ ]:
wandb.init(project=wandb_project)  ## 내가 만든 프로젝트 이름을 넣어주어야 합니다.

# 데이터 및 하이퍼파라미터 설정 
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

wandb.config.epochs = 10
wandb.config.batch_size = 64

# 모델을 구축합니다
model = Sequential() ## 과제시에는 이 모델을 Tre-trained model로 대체하면 됩니다. 
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


In [ ]:
# 모델학습방식을 정의함
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습시키기
model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

성능이 마음에 안든다면 추가로 학습을 더 시키는 방법도 있습니다. 

In [ ]:
wandb.config.epochs = 20
wandb.config.batch_size = 512

model.fit(train_images, train_labels, 
          validation_data=(test_images, test_labels),
          epochs=wandb.config.epochs, 
          batch_size=wandb.config.batch_size, 
          callbacks=[WandbCallback()]
          )

In [ ]:
!ls wandb/